请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 

# 导入所需要的包

In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

# 导入数据，数据处理

In [2]:
#导入数据
data=pd.read_csv("决策树.csv")
#需要进行独热编码的列
oneHotColumns=["性别","父母鼓励"]
#创建独热编码对象
oneHotEncoder=OneHotEncoder()
#训练热度编码器
oneHotEncoder.fit(data[oneHotColumns])
#转换数据为独热编码，二维数组
oneHotData=oneHotEncoder.transform(data[oneHotColumns]).toarray()
oneHotData
#把二维数组转DataFrame，oneHotEncoder.get_feature_names()获取独热编码器训练数据后所产生的列名
df_onehotData=pd.DataFrame(oneHotData,columns=oneHotEncoder.get_feature_names())
df_onehotData
#把独热编码DataFrame与"父母收入","IQ"的Dataframe拼接
x=df_onehotData.join(data[["父母收入","IQ"]])
y=data["升学计划"]
x
print(x.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x0_女    8000 non-null   float64
 1   x0_男    8000 non-null   float64
 2   x1_不鼓励  8000 non-null   float64
 3   x1_鼓励   8000 non-null   float64
 4   父母收入    8000 non-null   int64  
 5   IQ      8000 non-null   int64  
dtypes: float64(4), int64(2)
memory usage: 375.1 KB
None


# 创建决策树，并K折交叉验证

In [8]:
from sklearn.model_selection import GridSearchCV

In [11]:
#网络搜索，寻找最优参数
paramGrid=dict(
    max_depth=range(4,20),   #0-19
    max_leaf_nodes=range(4,20)
)
dtModel=DecisionTreeClassifier()
#创建网络搜索对象，设置参数
grid=GridSearchCV(
    dtModel,param_grid=paramGrid,
    cv=10,return_train_score=True,
    n_jobs=10
)
#开始训练
grid=grid.fit(x,y)
#打印最优准确率
print(grid.best_score_)
#打印最优的超参数
print(grid.best_params_)


0.8391250000000001
{'max_depth': 6, 'max_leaf_nodes': 14}


In [15]:
#创建决策树的深度为3，最大叶子节点数为7
dtModel=DecisionTreeClassifier(
    criterion="entropy",
    max_depth=6,
    max_leaf_nodes=14
)
cvs=cross_val_score(dtModel,x,y,cv=10)
cvs.mean()

0.835375

# 决策树图形化

In [4]:
# 安装pydot_ng
!pip install pydot_ng

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [5]:
import pydot_ng as pydot
from sklearn.externals.six import StringIO
from sklearn.tree import export_graphviz

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


In [6]:

#训练模型
dtModel.fit(x,y)
#创建StringIO,准备把dot写入StringIO
dot_data=StringIO()
#获取x列名
f=list(x.columns)
#把决策树转为dot文件写入StringIO
export_graphviz(
    dtModel,
    out_file=dot_data,
    class_names=["不计划","计划"],
    feature_names=f,
    filled=True,rounded=True,special_characters=True
)
#把dot转png
graph=pydot.graph_from_dot_data(
    dot_data.getvalue()
)
graph.write_png(
    "opt.png"
)

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.549478 to fit



True

# 预测

In [25]:
px=pd.DataFrame({
    "性别":["男"],
    "父母收入":[40000],
    "IQ":[120],
    "父母鼓励":["鼓励"],
    "升学计划":["不计划"]
})
#使用上面训练好的独热编码器，对性别、父母鼓励进行独热编码
px_oneHotData=oneHotEncoder.transform(px[["性别","父母鼓励"]]).toarray()
px_oneHotData
#把独热编码后的数组转为DataFrame
df_px_oneHotData=pd.DataFrame(px_oneHotData,columns=oneHotEncoder.get_feature_names())
df_px_oneHotData
#加入"父母收入","IQ"
px=px[["父母收入","IQ"]].join(df_px_oneHotData)
px
#使用训练好的决策树模型进行预测
r=dtModel.predict(px)
r

array(['不计划'], dtype=object)